# Large file processor
Aim is to build a system which is able to handle long running processes in a distributed fashion.

# 1. Problem statement
We need to be able to import products from a CSV file and into a database. There are half a million product details to be imported into the database. You can find the CSV file here in a compressed format

# 2. Points to achieve
1. Your code should follow concept of OOPS
2. Support for regular non-blocking parallel ingestion of the given file into a table. Consider thinking about the scale of what should happen if the file is to be processed in 2 mins.
3. Support for updating existing products in the table based on `sku` as the primary key. (Yes, we know about the kind of data in the file. You need to find a workaround for it)
4. All product details are to be ingested into a single table
5. An aggregated table on above rows with `name` and `no. of products` as the columns

# Process
Since code should have the concept of Object-Oriented Programming.
Here I am explaining the process/steps that I followed to write the code.
1.	I wrote code is SSMS to check the things how it is working.
2.	I converted same code to Python and checked the workflow and it worked fine.
3.	Since my code is not reaching the first point in the Point to Achieve, I modified code in such a way that it should satisfy the OOPS concept. I have created classes and Object and required functions as well.


In [18]:
#Here I created clasee called Products which required fuctions


# init fuction will work as connection function where it will connected to the Server and Database.
# Made conn.autocommit=True so we don't need to call the commit() method manually to commit the changes.
# Passing Server and database name as a parameter.
import pyodbc

class Products:
  def __init__(self,Server, Database):
    
    conn = pyodbc.connect('Driver={SQL Server};'+
                      'Server={0};'.format(Server)+
                      'Database={0};'.format(Database)+
                      'Trusted_Connection=yes;')
    conn.autocommit=True
    self.cursor = conn.cursor()
    
# Here I am creating a function called create_table and passing table name as parameter with the help of format function

  def create_table(self,table):
    self.cursor.execute('CREATE TABLE {0} (name nvarchar(MAX), sku nvarchar(MAX), description nvarchar(MAX))'.format(table))
    
    
# Here I am creating a function called drop_drop the table if it is called created. This is an "Optional"

  def drop_table(self,table):
    self.cursor.execute('DROP TABLE IF EXISTS {0}'.format(table))
    
    
# Here I a, creating a function to insert the data to a table  from CSV file. This function will take table name and file path as a parameter.

  def bulk_insert(self,table,file_path):
    self.cursor.execute("""BULK INSERT {0} FROM "{1}" WITH ( FIELDTERMINATOR=',',ROWTERMINATOR='\\n');""".format(table,file_path))
    
# Here is the function to update the existing table

  def update_by_sku(self,table,product,description,sku):
    # print('UPDATE MYPRODUCTS1 SET name='testproduct1', description='testdescription1' WHERE sku='lay-raise-best-end'.format(table,product,description,sku))
    self.cursor.execute("UPDATE {0} SET name='{1}', description='{2}' WHERE sku='{3}'".format(table,product,description,sku))

In [19]:
#Here I am passing/calling actual Servername and Database name
p1 = Products("####\####", "Product") #Please reply Servername with actual Servername as I cannot disclose my Servername

In [20]:
#Here I created 2 variables calledd table name and file path and assigned actual values
table_name="MYPRODUCTS1"
file_path="C:\\Users\\chand\\Desktop\\Project1\\products.csv" #Please reply File path with actual file path as I cannot disclose my file path

In [21]:
# Here I am caling the fucntions
p1.drop_table(table_name)
p1.create_table(table_name)
p1.bulk_insert(table_name,file_path)
# Below I am calling the function to update the records
p1.update_by_sku(table_name,"testproduct1","testdescription1","lay-raise-best-end")

In [22]:
#Here is the query to get a aggregated table
#SELECT d.name, COUNT(1) AS 'no. of products'
#INTO dbo.AggregatedProducts
#FROM dbo.MYPRODUCTS1 d
#GROUP BY d.name


#SELECT * FROM  dbo.AggregatedProducts